In [ ]:
from dask.distributed import Client, LocalCluster
from dask import dataframe as dd

cluster = LocalCluster(
    n_workers=2,
    threads_per_worker=2,
    memory_limit='4GB'  
)
client = Client(cluster)


c:\Users\natal\PycharmProjects\Large-scale-dataset-processing-using-Dask\.conda\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50333 instead
  warnings.warn(
c:\Users\natal\PycharmProjects\Large-scale-dataset-processing-using-Dask\.conda\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.51s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Dask client connected
<Client: 'tcp://127.0.0.1:50337' processes=2 threads=4, memory=3.73 GiB>


2025-06-11 01:16:33,237 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:50348 (pid=14908) exceeded 95% memory budget. Restarting...
2025-06-11 01:16:33,866 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50348' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('read_parquet-103a5c4fbd3be517d41db400c7e7a2ae', 3), ('read_parquet-103a5c4fbd3be517d41db400c7e7a2ae', 5), ('chunk-4c44b446e9669427487472b80f5fc66c', 1)} (stimulus_id='handle-worker-cleanup-1749597393.8647346')
2025-06-11 01:16:34,509 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:50351 (pid=9860) exceeded 95% memory budget. Restarting...
2025-06-11 01:16:34,600 - distributed.nanny - WARNING - Restarting worker
2025-06-11 01:16:34,695 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50351' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('read_parquet-103a5c4fbd3be517d41db400c7e7

In [9]:
print("Dask client connected")
print(client.dashboard_link)

Dask client connected
http://127.0.0.1:50333/status


In [26]:
df = dd.read_parquet("../data/yellow_tripdata_2024-*.parquet", blockszie="65MB")

In [27]:
n_rows = df.shape[0].compute()
n_cols = df.shape[1]
print(f"{n_rows:,} rows × {n_cols} columns")


41,169,720 rows × 19 columns


In [28]:
print("Columns:", df.columns)
print("Partition number:", df.npartitions)

Columns: Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')
Partition number: 12


In [29]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [30]:
df['month'] = df['tpep_pickup_datetime'].dt.month
df['trip_distance_km'] = df['trip_distance'] * 1.60934

In [31]:
courses_per_month = df.groupby('month').size().compute()

In [32]:
courses_per_month

month
1     2964637
2     3007534
3     3582613
4     3514295
5     3723843
12    3668381
6     3539172
7     3076876
8     2979192
9     3633025
10    3833780
11    3646372
dtype: int64

In [33]:
df["passenger_count"].value_counts().compute()

passenger_count
3.0     1282073
5.0      320613
7.0          56
8.0         192
0.0      401354
1.0    28632703
2.0     5410774
4.0      814889
6.0      215798
9.0          36
Name: count, dtype: int64

In [34]:
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df['trip_duration'].mean().compute()


np.float64(17.46790819887367)